<a href="https://colab.research.google.com/github/MatthewGuile/tool-lyrics-nlp/blob/main/tool_lyrics_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

In [1]:
import json 
import requests

url = "https://raw.githubusercontent.com/MatthewGuile/tool-lyrics-nlp/main/files/Lyrics_Tool.json"
resp = requests.get(url)
data = json.loads(resp.text)

In [2]:
data.keys()

dict_keys(['alternate_names', 'api_path', 'description', 'facebook_name', 'followers_count', 'header_image_url', 'id', 'image_url', 'instagram_name', 'is_meme_verified', 'is_verified', 'name', 'translation_artist', 'twitter_name', 'url', 'current_user_metadata', 'description_annotation', 'user', 'songs'])

In [22]:
import pandas as pd

# load data as a pandas dataframe
df0 = pd.DataFrame(data['songs'])

In [4]:
df0.keys()

Index(['annotation_count', 'api_path', 'artist_names', 'full_title',
       'header_image_thumbnail_url', 'header_image_url', 'id',
       'lyrics_owner_id', 'lyrics_state', 'path', 'pyongs_count',
       'song_art_image_thumbnail_url', 'song_art_image_url', 'stats', 'title',
       'title_with_featured', 'url', 'primary_artist', 'apple_music_id',
       'apple_music_player_url', 'description', 'embed_content',
       'featured_video', 'lyrics_placeholder_reason', 'recording_location',
       'release_date', 'release_date_for_display', 'current_user_metadata',
       'album', 'custom_performances', 'description_annotation',
       'featured_artists', 'lyrics_marked_complete_by', 'media',
       'producer_artists', 'song_relationships', 'verified_annotations_by',
       'verified_contributors', 'verified_lyrics_by', 'writer_artists',
       'artist', 'lyrics'],
      dtype='object')

In [5]:
# select relevant columns 
df = df0[['title', 'lyrics', 'release_date']]

In [30]:
# scrape albums and create column 
album_list = []

for i in range(len(df0['album'])):
    try:
        album_list.append(df0['album'][i]['name'])
    except:
        album_list.append(None)
        continue

df['album'] = album_list


In [31]:
# select the albums required for analysis and drop irrelevant data
relevant_albums = ['Opiate', 'Undertow', 'Ænima', '10,000 Days', 'Fear Inoculum']

df = df[df['album'].isin(relevant_albums)]